In [1]:
import requests as rq
import pymongo as pm
import ujson as js
import pandas as pd
import numpy as np
from datetime import datetime as dt

from ApiHandler import ApiHandler

In [2]:
class ApiAssets(ApiHandler):
    settings = {
        **ApiHandler.settings,
        'valid_name_type_ids': [
            3465, 3466, 3467, 11488, 11489, 17363, 17364, 17365, 24445, 33003, 33005, 33007, 33009, 33011, 3293, 3296, 3297, ## Containers
            17366, 17367, 17368, 35825, 35826, 35827, 35835, 35836, 35832, 35833, 35834, 40340, 47512, 47513, 47514, 47515, 47516 ## Upwell Stations
        ]
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/corporations/{corp_id}/assets/',
        'names': 'https://esi.evetech.net/latest/corporations/{corp_id}/assets/names/'
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'Corp_Assets',
    }
    name = 'ESI Assets API'
    auth = True
    delete = True
    
    def build_data(self, raw_data):
        data_frame = pd.concat([pd.DataFrame(js.loads(data)) for data in raw_data], axis=0, sort=False)
        data_frame['is_blueprint_copy'] = data_frame['is_blueprint_copy'].fillna(False)
        name_data = self.build_name_parse(data_frame)
        data_frame['item_name'] = data_frame['item_id'].map(name_data)
        data_frame['record_time'] = self.timestamp
        return data_frame
    
    def build_name_parse(self, data_frame):
        nameable_item_ids = data_frame.loc[
            data_frame['is_singleton'] & data_frame['type_id'].isin(api.settings['valid_name_type_ids']),
            'item_id'
        ].reset_index(drop=True)
        
        name_data = []
        name_splits = len(nameable_item_ids) // 1000
        for i in range(name_splits):
            name_data.append(nameable_item_ids[1000 * i: 1000 * (i+1)].astype(str))
        name_data.append(nameable_item_ids[1000 * name_splits:].astype(str))
        
        name_data = self.get_name_data(name_data)
        name_data = pd.concat([pd.DataFrame(js.loads(name_list)) for name_list in name_data], axis=0).set_index('item_id')['name']
        
        return name_data
    
    def get_name_data(self, name_item_ids):
        if self.verbose: self._verbose('get_name_data', 'Getting name data...')
        
        if self.auth:
            self.headers['Authorization'] = 'Bearer {token}'.format(**{
                'token': self.auth_token
            })
            
        named_items = []
        for item_ids in name_item_ids:
            data='[%s]' % ','.join(item_ids)
            data_conn = self.url_post(
                self.url['names'].format(**self.url_params),
                self.headers,
                self.params,
                data
            )
            named_items.append(data_conn.content)

        if self.verbose: self._verbose('get_name_data', 'Name data acquired.')
        
        return named_items
    
    def url_post(self, url, headers, params, data):
        data_conn = rq.post(
            url,
            headers=headers,
            params=params,
            data=data
        )
        
        if data_conn.status_code != 200:
            raise Exception("""\
                Connection returned a {status} code on pull.
                Message body:
                {body}\
            """.format(**{
                'status': data_conn.status_code,
                'body': data_conn.content.decode('utf-8')
            }))
            
        return data_conn

In [3]:
api = ApiAssets()
api.run_process()